In [1]:
import os
import sys
import numpy as np
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from importlib import reload
import math
from IPython.display import display, Markdown
from datetime import datetime

sys.path.insert(0, os.path.abspath('../tesi_sabella'))
# pd.options.display.max_rows = 999

import pyfluxc.pyfluxc as flux
import cicids2017_data_generator as generator
import data_generator as dt
reload(flux)
reload(generator)

<module 'cicids2017_data_generator' from '/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/tesi_sabella/cicids2017_data_generator.py'>

In [2]:
# Functions ----- #
def mprint(s):
    display(Markdown(s))
    
def to_dt(x):
    return pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%S.%f')

In [3]:
# Constants ----- #
BUCKET = 'CIC_IDS_2017_Friday'
PULL_EVERY_X_MINUTES = 15
PULL_DELTA_TIME = pd.Timedelta(minutes=PULL_EVERY_X_MINUTES)

TCPREPLAY_START = {
    "Monday": "2020-04-04T17:19:06.730097",
    "Tuesday": "2020-04-05T14:09:30.933025",
    "Wednesday": "2020-04-06T08:46:54.338920",
    "Thursday": "2020-04-07T08:08:37.127602808",
    "Friday": "2020-04-07T22:41:05.814784058"}
TCPREPLAY_START = { k: pd.to_datetime(s, format='%Y-%m-%dT%H:%M:%S.%f') for k, s in TCPREPLAY_START.items() }
    
CIC_IDS_2017_TIMES = {
    "Monday": ("2017-07-03T08:55:58.598308", "2017-07-03T17:01:34.472889"),
    "Tuesday": ("2017-07-04T08:53:32.364079", "2017-07-04T17:00:31.076755"),
    "Wednesday": ("2017-07-05T08:42:42.084372", "2017-07-05T17:10:19.780725"),
    "Thursday": ("2017-07-06T08:58:58.492265", "2017-07-06T17:04:44.364012"),
    "Friday": ("2017-07-07T08:59:39.599128", "2017-07-07T17:02:41.169108"),
}
CIC_IDS_2017_TIMES = { k: (to_dt(s), to_dt(e)) for k, (s,e) in CIC_IDS_2017_TIMES.items() }

# Generating time series

We pull only the time range in which samples are available

In [6]:
fclient = flux.FluxClient(); 
start, end = fclient.bucket_timerange(BUCKET)
print(f"**first sample**: {start}\n**last sample**: {end}\nTotal: {end-start}")

**first sample**: 2020-04-07 22:40:35
**last sample**: 2020-04-08 07:10:55
Total: 0 days 08:30:20


In [7]:
cicids2017 = generator.CICIDS2017(BUCKET, '15s', fclient, start)

Extract data from influxdb in time intervals due to influx kill signal

In [8]:
i = 1
pull_start = start
ts = []
while True:
    pull_end = pull_start + PULL_DELTA_TIME
    
    print("------------")
    print(f"first sample: {pull_start}\nlast sample: {pull_end}\nTotal: {pull_end-pull_start}")
    print("------------")
    
    # TODO: return tuple
    t_now = datetime.now()
    pull_start, new_samples = cicids2017.pull(stop=pull_end)
    ts.append(datetime.now() - t_now)
    if new_samples is None:
        break

mprint(f"**Mean pulling time:** {np.mean(ts)}")

------------
first sample: 2020-04-07 22:40:35
last sample: 2020-04-07 22:55:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-07 22:55:35
last sample: 2020-04-07 23:10:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-07 23:10:35
last sample: 2020-04-07 23:25:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-07 23:25:35
last sample: 2020-04-07 23:40:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-07 23:40:35
last sample: 2020-04-07 23:55:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-07 23:55:35
last sample: 2020-04-08 00:10:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-08 00:10:35
last sample: 2020-04-08 00:25:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-08 00:25:35
last sample: 2020-04-08 00:40:35
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-08 00:40:35
last sample: 2020-04-08 0

**Mean pulling time:** 0:01:07.093571

In [9]:
cicids2017_df = cicids2017.to_pandas()

# Setting correct timing before saving
day = BUCKET.split("_")[-1]
delta = TCPREPLAY_START[day] - CIC_IDS_2017_TIMES[day][0]
new_delta_time = (cicids2017_df.index.levels[2] - delta)
cicids2017_df.index.set_levels(new_delta_time, level="_time", inplace=True)

cicids2017_df.to_pickle(f"./{BUCKET}.pkl")

# Dataframe Merging